# Get stock info (HistPrice, Trend)

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime
import io , os , time ,requests
from yahoofinancials import YahooFinancials
from time import sleep
import json
from finta import TA

In [3]:
# init 
datapath = '/mnt/c/Users/poom/drive/01project/01cap_stone/04data/01stockprice'

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# create list of S&P500 symbols
symbol = pd.read_csv(os.path.join(datapath,"sp500symbol.csv"))
symbol_list = symbol['Symbol'].tolist()

for sym in enumerate(symbol_list):
    if '.' in sym[1]:
        del symbol_list[sym[0]]

## Get Historical Stock Price

In [14]:
# create empty dataframe
stock_final = pd.DataFrame()
start = datetime.datetime(2015,1,1)
end = datetime.datetime(2020,12,31)
# iterate over each symbol
for i in symbol_list:  
    
    # print the symbol which is being downloaded
    # print( str(symbol_list.index(i)) + str(' : ') + i, sep=',', end=',', flush=True)  
    
    try:
        # download the stock price 
        stock = []
        stock = yf.download(i,start=start, end=end, progress=False)
        
        # append the individual stock prices 
        if len(stock) == 0:
            None
        else:
            stock['Name']=i
            stock_final = stock_final.append(stock,sort=False)
    except Exception:
        None


1 Failed download:
- BRK.B: No data found, symbol may be delisted

1 Failed download:
- BF.B: No data found for this date range, symbol may be delisted


In [15]:
stock_final.head()

,Open,High,Low,Close,Adj Close,Volume,Name
Date,,,,,,,
2014-12-31,28.205000,28.282499,27.552500,27.594999,25.137676,165613600.0,AAPL
2015-01-02,27.847500,27.860001,26.837500,27.332500,24.898552,212818400.0,AAPL
2015-01-05,27.072500,27.162500,26.352501,26.562500,24.197117,257142000.0,AAPL
2015-01-06,26.635000,26.857500,26.157499,26.565001,24.199400,263188400.0,AAPL
2015-01-07,26.799999,27.049999,26.674999,26.937500,24.538729,160423600.0,AAPL


In [18]:
## save hist price to csv
stock_final.to_csv("../04data/01stockprice/histprice.csv")

In [26]:
print("number of unique stock",len(stock_final['Name'].value_counts()))

number of unique stock 503


## Import Industry data 

In [29]:
stock_final = pd.read_csv("../04data/01stockprice/histprice.csv")

In [28]:
test = yf.Ticker("MA")
print(test.info['industry'])

Credit Services


In [55]:
# init dict
industry_map = {}
count = 0

for symbol in symbol_list:
    try:
        industry_map[symbol] = yf.Ticker(symbol).info['industry']
    except Exception:
        None
    count += 1
    if count % 40 == 0:
      sleep(5)
    

In [56]:
#save industry list as json file
import json
with open ('industry.json','w') as fp:
    json.dump(industry_map,fp)

In [57]:
stock_final['industry'] = stock_final['Name'].map(industry_map)
stock_final.to_csv("../04data/01stockprice/histprice2.csv")
stock_final.sample(5)

,Date,Open,High,Low,Close,Adj Close,Volume,Name,industry
39733,2017-08-03,63.209999,63.730000,63.180000,63.520000,57.245705,7018800.0,MRK,Drug Manufacturers—General
289982,2015-06-26,13.887375,14.032698,13.746594,13.864669,11.505390,44778900.0,HPQ,Computer Hardware
442231,2019-04-30,94.959999,96.940002,94.779999,96.900002,91.199203,1486400.0,ETR,Utilities—Diversified
643668,2015-10-16,37.146667,37.599998,36.673332,37.173332,33.500481,929100.0,WRB,Insurance—Property & Casualty
619589,2016-07-01,36.919998,37.130001,36.750000,36.880001,31.700029,1599000.0,UDR,REIT—Residential


## Import Google trend data

In [5]:
# Import TrendReq and Setting
from pytrends.request import TrendReq
pytrends = TrendReq(hl='en-US', tz=360)

In [7]:
# test create code for getting monthly Google Trend Data Overtime for a single stock
kw_list = ['TSLA']
pytrends.build_payload(kw_list, cat=0, timeframe= 'all', geo='US',gprop='')
Tempdata = pd.DataFrame(pytrends.interest_over_time())
Tempdata = Tempdata.drop(columns= ['isPartial'])
print(kw_list)
print(Tempdata.tail(5))

['TSLA']
            TSLA
date            
2020-08-01    65
2020-09-01   100
2020-10-01    48
2020-11-01    49
2020-12-01    75


In [10]:
## Monthly Google Trend Data OverTime

## Create a blank dataframe with date as index for merging with data from TempData table later.

kw_list = ['NONE']
pytrends.build_payload(kw_list, cat=0, timeframe='all', geo='US', gprop='')  #'today 5-y'
TempData = pd.DataFrame(pytrends.interest_over_time())
GGTrendAllTimeMonthly = TempData.drop(columns=['isPartial','NONE'])



## Getting 'Monthly Google Trend Data OverTime' for the first 50 stocks in StockName
count = 0

for Stock in symbol_list :
   kw_list = [Stock]
   pytrends.build_payload(kw_list, cat=0, timeframe='all', geo='US', gprop='')  #'today 5-y'
   TempData = pd.DataFrame(pytrends.interest_over_time())
   TempData = TempData.drop(columns=['isPartial'])
   GGTrendAllTimeMonthly = pd.concat([GGTrendAllTimeMonthly, TempData], axis=1, join='inner')

   count += 1
   if count % 40 == 0:
      sleep(20)
#    print(kw_list)
   #print(GGTrendAllTimeMonthly.head(2))
   #print('------------------')

In [13]:
GGTrendAllTimeMonthly.describe()

,AAPL,MSFT,AMZN,FB,GOOGL,GOOG,TSLA,BRK.B,JNJ,JPM,...,UNM,FOX,GPS,SLG,FTI,XRX,HFC,UAA,UA,NWS
count,204.000000,204.000000,204.000000,204.000000,204.000000,204.000000,204.000000,204.000000,204.000000,204.000000,...,204.000000,204.000000,204.000000,204.000000,204.000000,204.000000,204.000000,204.000000,204.000000,204.000000
mean,38.446078,22.877451,17.598039,13.975490,42.112745,37.990196,8.975490,21.485294,32.323529,29.480392,...,63.823529,49.333333,37.313725,35.808824,51.318627,33.647059,42.666667,51.200980,55.980392,57.813725
std,18.267364,15.216328,21.689824,19.176504,23.216947,16.840308,15.023279,14.905772,13.156251,15.596027,...,15.097654,9.214823,14.678143,17.327836,15.952869,15.518397,19.089023,17.758195,22.395012,11.469154
min,1.000000,7.000000,0.000000,0.000000,1.000000,3.000000,0.000000,0.000000,7.000000,4.000000,...,36.000000,32.000000,16.000000,0.000000,16.000000,0.000000,19.000000,17.000000,23.000000,32.000000
25%,27.000000,12.000000,4.000000,0.000000,24.750000,23.000000,0.000000,12.000000,24.000000,20.750000,...,50.750000,45.000000,26.000000,25.000000,40.750000,25.000000,28.000000,37.000000,37.000000,49.000000
50%,36.000000,17.000000,7.000000,10.000000,38.000000,38.000000,1.000000,18.000000,30.000000,26.000000,...,63.000000,49.000000,33.000000,32.000000,46.000000,32.000000,36.000000,49.000000,50.500000,58.000000
75%,49.000000,30.000000,22.000000,17.000000,60.500000,51.000000,12.000000,26.250000,38.250000,35.000000,...,76.000000,54.000000,47.000000,41.250000,59.000000,40.000000,55.250000,64.000000,80.250000,65.000000
max,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000


## Melting / Unpivot the Dataframe

In [60]:
GGTrendAllTimeMonthlyData = pd.melt(GGTrendAllTimeMonthly, id_vars=['date'], var_name = 'Name', value_name = 'GGTrendAllTimeMonthly') #value_vars=[StockName]
GGTrendAllTimeMonthlyData.sample(4)

,date,Name,GGTrendAllTimeMonthly
70675,2011-08-01,AES,59
37141,2005-02-01,TT,81
18401,2007-06-01,AXP,27
88406,2010-03-01,BWA,38


In [62]:
GGTrendAllTimeMonthlyData.to_csv("../04data/01stockprice/GGTrendAllTimeMonthly.csv")

## Download All General Data

In [7]:
#change directory
os.chdir(os.path.join(datapath,"01world_indices"))
os.getcwd()

'/mnt/c/Users/poom/drive/01project/01cap_stone/04data/01stockprice/01world_indices'

In [73]:
General_data = pd.date_range('2014-12-31','2020-12-28')
General_data = pd.DataFrame({"Date" :General_data
                            ,"d_Year" :General_data.year
                            ,"d_Month":General_data.month
                            ,"d_Date" :General_data.day
                            ,"d_FirstDayOfMonth":General_data.is_month_start.astype('int')
                            ,"d_DayInWeek":General_data.dayofweek
                            ,"d_WeekInMonth":General_data.week
                            ,"d_WeekInYear": General_data.weekofyear })
General_data.head(4)

,Date,d_Year,d_Month,d_Date,d_FirstDayOfMonth,d_DayInWeek,d_WeekInMonth,d_WeekInYear
0,2014-12-31,2014,12,31,0,2,1,1
1,2015-01-01,2015,1,1,1,3,1,1
2,2015-01-02,2015,1,2,0,4,1,1
3,2015-01-03,2015,1,3,0,5,1,1


In [74]:
General_data.Date = General_data.Date.astype('str').str.split(' ').str[0]
General_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2190 entries, 0 to 2189
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Date               2190 non-null   object
 1   d_Year             2190 non-null   int64 
 2   d_Month            2190 non-null   int64 
 3   d_Date             2190 non-null   int64 
 4   d_FirstDayOfMonth  2190 non-null   int64 
 5   d_DayInWeek        2190 non-null   int64 
 6   d_WeekInMonth      2190 non-null   int64 
 7   d_WeekInYear       2190 non-null   int64 
dtypes: int64(7), object(1)
memory usage: 137.0+ KB


In [75]:
# get each file name
filelist = os.listdir().copy()
name_filelist = np.array([name.split('.') for name in filelist])[:,0]

for File in filelist:
    TempData = pd.read_csv(File)
    TempData.columns = File.split('.')[0] +TempData.columns
    General_data = pd.merge(General_data,TempData, left_on= "Date", right_on= File.split('.')[0]+"Date" )
    General_data = General_data.drop(File.split('.')[0]+"Date", 1 )
General_data.head()

,Date,d_Year,d_Month,d_Date,d_FirstDayOfMonth,d_DayInWeek,d_WeekInMonth,d_WeekInYear,COILOpen,COILHigh,...,SP500Low,SP500Close,SP500Adj Close,SP500Volume,USDCNYOpen,USDCNYHigh,USDCNYLow,USDCNYClose,USDCNYAdj Close,USDCNYVolume
0,2015-01-05,2015,1,5,0,0,2,2,52.610001,52.730000,...,2017.339966,2020.579956,2020.579956,3799120000,6.1961,6.2090,6.1961,6.1961,6.1961,0.0
1,2015-01-06,2015,1,6,0,1,2,2,50.000000,50.369999,...,1992.439941,2002.609985,2002.609985,4460110000,6.2084,6.2084,6.1988,6.2084,6.2084,0.0
2,2015-01-07,2015,1,7,0,2,2,2,48.000000,49.310001,...,2005.550049,2025.900024,2025.900024,3805480000,6.2020,6.2040,6.1976,6.2020,6.2020,0.0
3,2015-01-08,2015,1,8,0,3,2,2,48.779999,49.650002,...,2030.609985,2062.139893,2062.139893,3934010000,6.2040,6.2102,6.2035,6.2040,6.2040,0.0
4,2015-01-09,2015,1,9,0,4,2,2,48.919998,49.610001,...,2038.329956,2044.810059,2044.810059,3364140000,6.2035,6.2035,6.1972,6.2035,6.2035,0.0


In [82]:
# na 
General_data.isna().sum().sum()
# Fill NaN by using 'Forward Fill' since all columns with blanks are time series data
# Filling the NaN data with the data in previous record might be the most appropriate way


402

In [83]:
General_data = General_data.fillna(method = 'ffill', axis = 0 , limit = None)
# na 
General_data.isna().sum().sum()

0

In [84]:
General_data.to_csv('../General_data.csv')

## Download Financial indicator :

In [2]:
# List of symbols for technical indicators
INDICATORS = ['RSI','STOCH','CCI','ADX','AO','MOM','MACD','STOCHRSI','WILLIAMS','EBBP', 'UO']



In [5]:
datapath = '/mnt/c/Users/poom/drive/01project/01cap_stone/04data/01stockprice'
stock_final = pd.read_csv(os.path.join(datapath,'histprice2.csv'))

In [ ]:
stock_final.head()

In [14]:
import datetime
def stock_retrived(quote, n, inv):
  start = (datetime.date.today() - datetime.timedelta(n) )
  end = datetime.datetime.today() 

  # auto_adjust = auto adjusted price for OHLC 
  df = yf.download(quote, start=start, end=end, interval=inv, auto_adjust = True)

  df.columns = df.columns.str.lower()
  #print(data.head())

  # plot for previous 500 business days of adjusted 'OPEN' price 
  #tmp = df.iloc[-500:]
  #tmp['open'].plot()

  return df

#data = stock_retrived("AOT.BK", NUM_DAYS, INTERVAL)

In [16]:
df = stock_retrived('TSLA',30,'1d')

[*********************100%***********************]  1 of 1 completed


In [17]:
df

,open,high,low,close,volume
Date,,,,,
2020-11-30,602.210022,607.799988,554.510010,567.599976,63003100
2020-12-01,597.590027,597.849976,572.049988,584.760010,40382800
2020-12-02,556.440002,571.539978,541.210022,568.820007,47775700
2020-12-03,590.020020,598.969971,582.429993,593.380005,42552000
2020-12-04,591.010010,599.039978,585.500000,599.039978,29401300
2020-12-07,604.919983,648.789978,603.049988,641.760010,56309700
2020-12-08,625.510010,651.280029,618.500000,649.880005,64265000
2020-12-09,653.690002,654.320007,588.000000,604.479980,71291200
2020-12-10,574.369995,627.750000,566.340027,627.070007,67083200
